In [ ]:
# standard python utilities
import os
from os.path import join, dirname, basename
import glob
import sys
import time
import pandas as pd
import numpy as np

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
doc_dir = os.getcwd()
while os.path.basename(doc_dir) != 'Documents':
    doc_dir = os.path.dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = os.path.dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
gwfm_dir


In [ ]:
# New model domain 52.9 deg
m_domain = gpd.read_file(join(gwfm_dir,'DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp'))

# Load model grid as geopandas object
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')



In [ ]:
uzf_path = gwfm_dir+'/UZF_data'
soil_path = uzf_path+'/wss_gsmsoil_CA'
# Read in the soil map spatial data
soil_gpd = gpd.read_file(soil_path+'/spatial/gsmsoilmu_a_ca.shp')
soil_gpd = soil_gpd.to_crs('EPSG:32610')
# soil_gpd.plot()

In [ ]:
# Intersect state soil map with model domain
mb.crs = "EPSG:32610"
gpd_mb = gpd.overlay(soil_gpd, m_domain, how = 'intersection')
# gpd_mb.plot(column = 'MUKEY')

In [ ]:
# Read in files relevant to water content, sat K, and epsilon that correspond to the spatial data
por = pd.read_csv(soil_path+'/soildb_US_2003_Porosity.txt',sep=',', header = 0)
ksat = pd.read_csv(soil_path+'/soildb_US_2003_Ksat.txt',sep=',', header = 0)
psdi = pd.read_csv(soil_path+'/soildb_US_2003_PSDI.txt',sep=',', header = 0)
# Pull out representative values for porosity, saturated hydraulic conductivity and pore size distribution index
por = por.drop(labels = ['Porosity_Low', 'Porosity_High'], axis = 1)
ksat = ksat.drop(labels = ['Ksat_Low', 'Ksat_High'], axis = 1)
psdi = psdi.drop(labels = ['PSDI_Low', 'PSDI_High'], axis = 1)

In [ ]:
# Prepare the column name for joining spatial data with the reference data
gpd_mb = gpd_mb.rename(columns={'MUKEY':'mukey'})
# gpd_mb, por
# gpd_mb.join(por, on = 'mukey')
soil = por.merge(ksat, on = 'mukey', how = 'inner')
soil = soil.merge(psdi, on = 'mukey', how = 'inner')
# Calculate epsilon using the pore size distribution index equation from Brooks and Corey 1960
soil['EPS'] = (2 + 3*soil.PSDI_Rep)/soil.PSDI_Rep
gpd_mb.mukey = gpd_mb.mukey.values.astype('int64')
gpd_mb = gpd_mb.merge(soil, on = 'mukey')


In [ ]:
# Samples the points every 10 meters to match with the 100 meter grid
grid_uzf = gpd.sjoin(gpd_mb, grid_p,   how = "inner", op= "intersects")
# grid_uzf = gpd.overlay(grid_p, gpd_mb,  how = 'intersection')


In [ ]:
# Dissolves the points every 10 meters to the 100 meter spacing
# grid_uzf = grid_uzf.dissolve(by = 'node')
# length of 70,073 before dissolve, 66,000 after dissolve

In [ ]:
# Convert Ksat from micrometers/sec to m/d
grid_uzf.Ksat_Rep *= (86400*(1E-6))

In [ ]:
# grid_uzf.to_file(uzf_path+'/final_grid_uzf/griduzf.shp')
grid_uzf = gpd.read_file(uzf_path+'/final_grid_uzf/griduzf.shp')

In [ ]:
def fill_uzf(uzfvalues, grid_uzf):
    temp = np.zeros((nrow,ncol))
    temp[grid_uzf.row.values-1,grid_uzf.column.values-1] = uzfvalues
    return(temp)



In [ ]:

soilKs_array = fill_uzf(grid_uzf.Ksat_Rep, grid_uzf)
soiln_array = fill_uzf(grid_uzf.Porosity_R, grid_uzf)
soileps_array = fill_uzf(grid_uzf.EPS, grid_uzf)

np.savetxt(uzf_path+'/final_soilKs.tsv', soilKs_array, delimiter = '\t')
np.savetxt(uzf_path+'/final_soiln.tsv', soiln_array, delimiter = '\t')
np.savetxt(uzf_path+'/final_soileps.tsv', soileps_array, delimiter = '\t')

# soilKs_array = np.loadtxt(uzf_path+'/final_soilKs.tsv', delimiter = '\t')
# soiln_array = np.loadtxt(uzf_path+'/final_soiln.tsv', delimiter = '\t')
# soileps_array = np.loadtxt(uzf_path+'/final_soileps.tsv', delimiter = '\t')


In [ ]:
temp = np.zeros((nrow,ncol))
temp[grid_uzf.row.values-1,grid_uzf.column.values-1] = grid_uzf.Ksat_Rep.values
temp.shape
plt.imshow(uzf.finf.array[0,0,:,:])